<a href="https://colab.research.google.com/github/custom-hyper/CoinGeckoAPI/blob/main/CoinGecko_ohlc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Connect to FNS { display-mode: "code" }
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#@title Install Libraries { display-mode: "code" }
!pip install plotly
!pip install pytrends
!pip install pycoingecko
!pip install pandas
!pip install tweepy
!pip install requests
!pip install pipreqs
!pip install configparser
!pip install psycopg2


In [3]:
#@title Import Project  Dependencies { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import logging
import sys
from tqdm.notebook import tqdm
import os
import sqlite3 as sq
import psycopg2 as pg
import configparser
from sqlalchemy import create_engine
from pycoingecko import CoinGeckoAPI

from __future__ import division

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Setup SQLite


In [4]:
#@title  Connect { display-mode: "code" }
path = '/content/drive/MyDrive/workspace/pipelines/db/' #@param {type: 'string'
database = "db.sqlite" #@param {type: 'string'
conn_SQLite = sq.connect(path+database)

In [5]:
#@title   Show Available Tables { display-mode: "code" }
cursor = conn_SQLite.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('deFi_lama_total_TVL',),
 ('deFi_lama_tvl',),
 ('deFi_lama_general',),
 ('deFi_lama_id',),
 ('proprietary_Twitter_id',),
 ('proprietary_tweets_extract',),
 ('proprietary_Twitter_partnerships_announcements',),
 ('CoinGecko_ohlc',),
 ('proprietary_airtable_watch_all',),
 ('proprietary_Twitter_Partnership_ohlc',),
 ('proprietary_master_id',)]

In [6]:
#@title Load Token ID to SQLite { display-mode: "code" }
proprietary_master_id  = "proprietary_master_id"


df_proprietary_master_id = pd.read_sql('select * from {}'.format(proprietary_master_id), conn_SQLite)

df_proprietary_master_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68451 entries, 0 to 68450
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CoinGecko_id    68451 non-null  object
 1   prop_id         68451 non-null  object
 2   symbol          68451 non-null  object
 3   name            68451 non-null  object
 4   prop_timestamp  68451 non-null  int64 
 5   anon_id         68451 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.1+ MB


In [7]:
#@title ok { display-mode: "code" }
cg = CoinGeckoAPI()
CoinGecko_id = df_proprietary_master_id['CoinGecko_id']
frames = []

try:
  start = 1
  ends = 1000
  for i in tqdm(CoinGecko_id[start:ends]):

    a = cg.get_coin_ohlc_by_id(id=i, vs_currency='usd', days = 'max')

    df = pd.DataFrame(data=a, columns = ['Date','Open','High', 'Low', 'Close'])
    
    df['CoinGecko_id'] = str(i)
    
    frames.append(df)
    time.sleep(60/50)
except ConnectionError:
    time.sleep(60)
single = pd.concat(frames) #Careful, this line must be outside of the for loop
single['prop_timestamp'] = int( time.time_ns() / 1000 )

path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/ohlc/output/"
single.to_csv(path + 'CoinGecko_ohlc_{}-{}.csv'.format(start,ends))


  0%|          | 0/999 [00:00<?, ?it/s]

In [10]:
import os
import glob
import pandas as pd
import sys
from pathlib import Path




path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/ohlc/output/bulk/"


Cleaned_data_file = 'Bulk_CoinGecko_ohlc.csv'

df = pd.DataFrame()

extension = 'csv'
#dataset_location = os.chdir(r'D:\investing_programs\datasets\CoinMetrics')
all_filenames = []
all_filenames = [i for i in glob.glob('/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/ohlc/output/*.{}'.format(extension))]

#combine all files in the list
for i in all_filenames:
    df = pd.read_csv(i)#.read(),engine='openpyxl')   
    print(df)

combined_csv = df.append([pd.read_csv(f) for f in all_filenames])


combined_csv.to_csv( path + Cleaned_data_file, index=False, encoding='utf-8-sig')


print(combined_csv.head())




,Unnamed: 0,Date,Open,High,Low,Close,CoinGecko_id,prop_timestamp
0,0,1597795200000,11007.430000,11007.430000,11007.430000,11007.430000,0-5x-long-algorand-token,1650735067899339
1,1,1598140800000,10946.390000,11086.450000,10946.390000,11086.450000,0-5x-long-algorand-token,1650735067899339
2,2,1599436800000,9147.460000,9147.460000,9147.460000,9147.460000,0-5x-long-algorand-token,1650735067899339
3,3,1599782400000,8892.700000,8892.700000,8892.700000,8892.700000,0-5x-long-algorand-token,1650735067899339
4,4,1604102400000,7497.080000,7497.080000,7495.000000,7495.000000,0-5x-long-algorand-token,1650735067899339
...,...,...,...,...,...,...,...,...
111420,37,1649289600000,0.075862,0.076548,0.040121,0.057285,aspo-world,1650735067899339
111421,38,1649635200000,0.054122,0.054122,0.047756,0.048007,aspo-world,1650735067899339
111422,39,1649980800000,0.047717,0.049841,0.043242,0.049841,aspo-world,1650735067899339
111423,40,1650326400000,0.048167,0.048167,0.030951,0.031592,aspo-world,1650735067899339


In [12]:
#@title Load Token ID to SQLite { display-mode: "code" }

CoinGecko_ohlc = "CoinGecko_ohlc"

combined_csv.to_sql(CoinGecko_ohlc, conn_SQLite, if_exists='replace', index=False) # writes to file
conn_SQLite.commit()

# Just be sure any changes have been committed or they will be lost.

CoinGecko_ohlc = pd.read_sql('select * from {}'.format(CoinGecko_ohlc), conn_SQLite)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [13]:
CoinGecko_ohlc.head()

,Unnamed: 0,Date,Open,High,Low,Close,CoinGecko_id,prop_timestamp
0,0,1597795200000,11007.43,11007.43,11007.43,11007.43,0-5x-long-algorand-token,1650735067899339
1,1,1598140800000,10946.39,11086.45,10946.39,11086.45,0-5x-long-algorand-token,1650735067899339
2,2,1599436800000,9147.46,9147.46,9147.46,9147.46,0-5x-long-algorand-token,1650735067899339
3,3,1599782400000,8892.70,8892.70,8892.70,8892.70,0-5x-long-algorand-token,1650735067899339
4,4,1604102400000,7497.08,7497.08,7495.00,7495.00,0-5x-long-algorand-token,1650735067899339


In [14]:
CoinGecko_ohlc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222850 entries, 0 to 222849
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      222850 non-null  int64  
 1   Date            222850 non-null  int64  
 2   Open            222850 non-null  float64
 3   High            222850 non-null  float64
 4   Low             222850 non-null  float64
 5   Close           222850 non-null  float64
 6   CoinGecko_id    222850 non-null  object 
 7   prop_timestamp  222850 non-null  int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 13.6+ MB
